This work has been done by referencing the work of my Professor Priyank Thakkar Sir at https://www.kaggle.com/code/priyankdl/machine-translation-torchtext-seq2seq-gru

Importing the dependencies

In [ ]:
# Install all packages together
!pip install torchtext==0.15.2 torch==2.0.1 portalocker>=2.0.0 torchdata -U spacy

In [ ]:
# After the restart, download the language models
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 67.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#Change the link for the download of the iter

In [ ]:
#Download the iter

#Make the tokenizer

#Make a special iter for the vocab
#Make the vocab for both the languages

#re-inialise the iter

#mount the collate function that prepares the batches for the model

#initialise the optimiser and the loss function
#make the dataloaders
#make the models for the encoder and the decoder
#encoder
  #embed
  #dropout layer
  #gru layer
  #return hiddens

#loop for train one epoch
  #batch inputs to encoder
  #take the last hiddens for the batch

  #take input sequence to the decoder excluding the eos
  #feed the hiddens to the decoder and the inputs
  #returns the logits after passing through the linear layer

  #use the logits along with the ground truth that is excluding the sos to calculate the loss function
  #display the running loss
  #backprop


#loop for eval one epoch
  #take one sentence at a time

  #batch inputs to encoder
  #get the hidden

  #current_input=sos
  #loop for one sentence
    #pass hidden and sos to deocder
    #get logits after the linear layer and the hidden

    #do softmax on logits then argmax to find the index number
    #find the token and add to answer
    #if token == eos || loop for more than 256:
      #break

    #current_input=index number


#run train one epoch and eval one epoch and check the accuracy





In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torchtext
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import multi30k,Multi30k

from functools import partial

In [ ]:
batch_size=16
embed_size=300
hidden_size=512

In [ ]:
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

train_iter=Multi30k(
    split="train",
    language_pair=("de","en")
)

de_tokenizer=get_tokenizer("spacy","de_core_news_sm")
en_tokenizer=get_tokenizer("spacy","en_core_web_sm")

de_iter_for_vocab=[]
en_iter_for_vocab=[]

for (de,en) in train_iter:
  de_tokens=de_tokenizer(de)
  en_tokens=en_tokenizer(en)

  de_iter_for_vocab.append(de_tokens)
  en_iter_for_vocab.append(en_tokens)


de_vocab=build_vocab_from_iterator(
    de_iter_for_vocab,
    specials=["<sos>","<eos>","<pad>","<unk>"],
    special_first=True,
    min_freq=1
)

de_vocab.set_default_index(de_vocab["<unk>"])

en_vocab=build_vocab_from_iterator(
    en_iter_for_vocab,
    specials=["<sos>","<eos>","<pad>","<unk>"],
    special_first=True,
    min_freq=1
)

en_vocab.set_default_index(en_vocab["<unk>"])

print("The de vocab size is ",de_vocab.__len__())
print("The en vocab size is ",en_vocab.__len__())



The de vocab size is  19214
The en vocab size is  10837


In [ ]:
def text_pipeline(line,option): #option 1 for de and option 2 for en
  if (option==1):
    return de_vocab.lookup_indices(de_tokenizer(line))
  else:
    return en_vocab.lookup_indices(en_tokenizer(line))


In [ ]:
def collate_function(batch):

  inputs=[]
  gt=[]

  for de,en in batch:
    de_tokens=text_pipeline(de,1)
    en_tokens=text_pipeline(en,2)

    de_tokens.append(de_vocab["<eos>"])
    en_tokens.append(en_vocab["<eos>"])

    de_tokens.insert(0,de_vocab["<sos>"])
    en_tokens.insert(0,en_vocab["<sos>"])

    de_tokens=torch.tensor(de_tokens)
    en_tokens=torch.tensor(en_tokens)

    inputs.append(de_tokens)
    gt.append(en_tokens)

  inputs = pad_sequence(inputs, batch_first=True, padding_value=de_vocab["<pad>"])
  gt = pad_sequence(gt, batch_first=True, padding_value=en_vocab["<pad>"])

  return inputs,gt



In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_size,embed_size,hidden_size):
    super().__init__()
    self.embed=nn.Embedding(input_size,embed_size)
    self.dropout=nn.Dropout(0.1)
    self.gru=nn.GRU(embed_size,hidden_size,batch_first=True)

  def forward(self,x):
    x=self.embed(x)
    x=self.dropout(x)
    outputs,hidden=self.gru(x)

    return hidden

class Decoder(nn.Module):
  def __init__(self,input_size,embed_size,hidden_size):
    super().__init__()
    self.embed=nn.Embedding(input_size,embed_size)
    self.gru=nn.GRU(embed_size,hidden_size,batch_first=True)
    self.linear=nn.Linear(hidden_size,input_size)

  def forward(self,x,prev_hidden):
    x=self.embed(x)
    outputs,hidden=self.gru(x,prev_hidden)
    logits=self.linear(outputs)

    return logits,hidden

In [ ]:
encoder=Encoder(de_vocab.__len__(),embed_size,hidden_size)
decoder=Decoder(en_vocab.__len__(),embed_size,hidden_size)

In [ ]:
encoder.load_state_dict(torch.load("/content/encoder_state_dict.pth"))
decoder.load_state_dict(torch.load("/content/decoder_state_dict.pth"))

<All keys matched successfully>

In [ ]:
loss_function=nn.CrossEntropyLoss()
en_optim=optim.Adam(params=encoder.parameters(),lr=0.001)
de_optim=optim.Adam(params=decoder.parameters(),lr=0.001)

In [ ]:
#loop for train one epoch
  #batch inputs to encoder
  #take the last hiddens for the batch

  #take input sequence to the decoder excluding the eos
  #feed the hiddens to the decoder and the inputs
  #returns the logits after passing through the linear layer

  #use the logits along with the ground truth that is excluding the sos to calculate the loss function
  #display the running loss
  #backprop


def train_one_epoch():
  encoder.train()
  decoder.train()

  total_loss=0
  batches=0

  train_iter=Multi30k(
    split="train",
    language_pair=("de","en")
  )
  train_dataloader=DataLoader(train_iter,batch_size=batch_size,shuffle=True,collate_fn=collate_function)


  for inputs,gt in train_dataloader:
    last_hiddens=encoder(inputs) #1,16,512
    # print(inputs.shape)
    # print(gt.shape)
    # print(last_hiddens.shape)
    in_decoder=gt[:,:-1]
    # print(in_decoder.shape)
    logits,hidden=decoder(in_decoder,last_hiddens) #16,m-1,10837  1,16,512
    # print("Shape of Logits:",logits.shape)
    # print("Shape of hidden:",hidden.shape)
    y=gt[:,1:] #16,m-1

    logits=logits.reshape(-1,en_vocab.__len__()) #16*m-1,10837
    y=y.reshape(-1) #16*m-1

    loss=loss_function(logits,y)
    total_loss+=loss.item()
    batches+=1
    print("Running_loss:(Batch ",batches,"):",total_loss/batches)

    en_optim.zero_grad()
    de_optim.zero_grad()

    loss.backward()

    en_optim.step()
    de_optim.step()



In [ ]:
#loop for eval one epoch
  #take one sentence at a time

  #batch inputs to encoder
  #get the hidden

  #current_input=sos
  #loop for one sentence
    #pass hidden and sos to deocder
    #get logits after the linear layer and the hidden

    #do softmax on logits then argmax to find the index number
    #find the token and add to answer
    #if token == eos || loop for more than 256:
      #break

    #current_input=index number


def eval_one_epoch():

  encoder.eval()
  decoder.eval()

  total_loss=0
  batches=0

  test_iter=Multi30k(
    split="valid",
    language_pair=("de","en")
  )
  test_dataloader=DataLoader(test_iter,batch_size=1,shuffle=True,collate_fn=collate_function)

  for inputs,gt in test_dataloader: #1,m  1,n


    german_sentence=de_vocab.lookup_tokens(inputs[0].tolist())
    english_sentence=en_vocab.lookup_tokens(gt[0].tolist())
    last_hidden=encoder(inputs) #1,1,512

    current_input=gt[:,0:1] #1,1
    last_a=last_hidden

    answer=""

    for i in range(len(gt[0])-1):
      logits,hidden=decoder(current_input,last_a) #1,1,10837  1,1,512
      logits=logits.squeeze(0)

      loss=loss_function(logits,gt[:,i+1].reshape(-1))
      total_loss+=loss.item()

      softmax=nn.Softmax(dim=1)
      logits=softmax(logits)  #1,10837

      predicted_index=torch.argmax(logits,dim=1)  #1,
      predicted_index=predicted_index[0].item() #int

      predicted_token=en_vocab.lookup_tokens([predicted_index])[0]

      answer+=predicted_token
      answer+=" "

      if (predicted_token=="<eos>"):
        break

      current_input=torch.tensor([[predicted_index]])
      last_a=hidden



    batches+=1


    print()
    print("--------------------------------------")
    print("Running_loss:(Batch ",batches,"):",total_loss/batches)
    print("DE: ",german_sentence)
    print("EN: ",english_sentence)
    print("Predicted-EN: ",answer)
    print("--------------------------------------")
    print()


  # print("Demo for the last answer:",answer)



In [ ]:
train_one_epoch()

Running_loss:(Batch  1 ): 9.193641662597656
Running_loss:(Batch  2 ): 8.76254415512085
Running_loss:(Batch  3 ): 8.30606476465861
Running_loss:(Batch  4 ): 8.025187849998474
Running_loss:(Batch  5 ): 7.731022548675537
Running_loss:(Batch  6 ): 7.233508189519246
Running_loss:(Batch  7 ): 6.976968560900007
Running_loss:(Batch  8 ): 6.612945079803467
Running_loss:(Batch  9 ): 6.568723731570774
Running_loss:(Batch  10 ): 6.344012451171875
Running_loss:(Batch  11 ): 6.163187807256525
Running_loss:(Batch  12 ): 6.032607793807983
Running_loss:(Batch  13 ): 5.902809326465313
Running_loss:(Batch  14 ): 5.800940990447998
Running_loss:(Batch  15 ): 5.6880108197530115
Running_loss:(Batch  16 ): 5.535694018006325
Running_loss:(Batch  17 ): 5.376224615994622
Running_loss:(Batch  18 ): 5.2785218424267235
Running_loss:(Batch  19 ): 5.2287538804506
Running_loss:(Batch  20 ): 5.165716743469238
Running_loss:(Batch  21 ): 5.10471384865897
Running_loss:(Batch  22 ): 5.018782756545327
Running_loss:(Batch  2

In [ ]:
torch.save(encoder.state_dict(), "encoder_state_dict.pth")

In [ ]:
torch.save(decoder.state_dict(), "decoder_state_dict.pth")

In [ ]:
torch.save(en_vocab,"en_vocab.pth")
torch.save(de_vocab,"de_vocab.pth")

In [ ]:
eval_one_epoch()

Streaming output truncated to the last 5000 lines.

--------------------------------------
Running_loss:(Batch  391 ): 86.22299588052795
DE:  ['<sos>', 'Eine', 'warm', 'gekleidete', 'Frau', 'in', 'Schwarz', 'kniet', 'mit', 'einem', 'kleinen', 'hellbraunen', 'Hund', 'bei', 'einer', 'Zuschauermenge', '.', '<eos>']
EN:  ['<sos>', 'A', 'warmly', 'dressed', 'woman', 'in', 'black', 'kneels', 'with', 'a', 'small', 'tan', 'dog', 'near', 'a', 'crowd', 'of', 'onlookers', '.', '<eos>']
Predicted-EN:  A woman in a black shirt is walking down a street with a shopping bag . <eos> 
--------------------------------------


--------------------------------------
Running_loss:(Batch  392 ): 86.13341419909467
DE:  ['<sos>', 'Drei', 'Kinder', 'stehen', 'nahe', 'beieinander', 'an', 'einem', 'hohen', 'blauen', 'Holzpfahl', '.', '<eos>']
EN:  ['<sos>', 'Three', 'children', 'standing', 'near', 'each', 'other', 'and', 'next', 'to', 'a', 'tall', 'blue', 'wooden', 'post', '.', '<eos>']
Predicted-EN:  Three child

In [ ]:
eval_one_epoch()

Running_loss:(Batch  1 ): 137.76415870338678
['<sos>', 'Zwei', 'Menschen', 'in', 'einfarbigen', '<unk>', 'aus', 'Elasthan', 'sitzen', 'auf', 'Stühlen', 'in', 'einem', 'Publikum', '.', '<eos>']
['<sos>', 'Two', 'people', 'in', 'full', 'body', 'solid', 'color', 'spandex', 'suits', 'sit', 'in', 'chairs', 'in', 'an', 'audience', '.', '<eos>']
Two people in a white shirt , one in a red shirt , one in a red 
Demo for the last answer: Two people in a white shirt , one in a red shirt , one in a red 
